In [1]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data

In [2]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(dataset_cora.num_node_features, 16, 'lstm')
        self.conv2 = SAGEConv(16, dataset_cora.num_classes, 'lstm')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [5]:
model = Net()
print(model)

Net(
  (conv1): SAGEConv(1433, 16, aggr=lstm)
  (conv2): SAGEConv(16, 7, aggr=lstm)
)


In [6]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

cuda:1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [13]:
for i in data.edge_index[1]:
    print(i)

tensor(0, device='cuda:1')
tensor(0, device='cuda:1')
tensor(0, device='cuda:1')
tensor(1, device='cuda:1')
tensor(1, device='cuda:1')
tensor(1, device='cuda:1')
tensor(2, device='cuda:1')
tensor(2, device='cuda:1')
tensor(2, device='cuda:1')
tensor(2, device='cuda:1')
tensor(2, device='cuda:1')
tensor(3, device='cuda:1')
tensor(4, device='cuda:1')
tensor(4, device='cuda:1')
tensor(4, device='cuda:1')
tensor(4, device='cuda:1')
tensor(4, device='cuda:1')
tensor(5, device='cuda:1')
tensor(5, device='cuda:1')
tensor(5, device='cuda:1')
tensor(6, device='cuda:1')
tensor(6, device='cuda:1')
tensor(6, device='cuda:1')
tensor(6, device='cuda:1')
tensor(7, device='cuda:1')
tensor(8, device='cuda:1')
tensor(8, device='cuda:1')
tensor(8, device='cuda:1')
tensor(9, device='cuda:1')
tensor(9, device='cuda:1')
tensor(10, device='cuda:1')
tensor(10, device='cuda:1')
tensor(11, device='cuda:1')
tensor(11, device='cuda:1')
tensor(12, device='cuda:1')
tensor(12, device='cuda:1')
tensor(12, device='cud

In [7]:
import torch.optim as optim

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [9]:
model.train()
for epoch in range(200):
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    acc = correct/data.train_mask.sum().item()

    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))

Epoch 000 train_loss: 1.9464 train_acc: 0.1643
Epoch 001 train_loss: 1.9395 train_acc: 0.2500
Epoch 002 train_loss: 1.9254 train_acc: 0.2643
Epoch 003 train_loss: 1.9100 train_acc: 0.2500
Epoch 004 train_loss: 1.8944 train_acc: 0.2643
Epoch 005 train_loss: 1.8738 train_acc: 0.5214


KeyboardInterrupt: 